# Eye of ML
Object detection with video cam written in Python

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
from tqdm import tqdm

import torch
from torch import nn
from torch.backends import cudnn

# cudnn.fastest = True
cudnn.benchmark = True

In [53]:
# cv2.namedWindow("preview")
vc = cv2.VideoCapture(0)

In [54]:
if vc.isOpened():
    rval, frame = vc.read()
else:
    rval = False

In [55]:
vc.release()
# cv2.destroyWindow("preview")

See how our frame looks like. 

In [ ]:
plt.imshow(frame)

In [21]:
??cv2.VideoCapture.read

Docstring:
read([, image]) -> retval, image
.   @brief Grabs, decodes and returns the next video frame.
.   
.       @param [out] image the video frame is returned here. If no frames has been grabbed the image will be empty.
.       @return `false` if no frames has been grabbed
.   
.       The method/function combines VideoCapture::grab() and VideoCapture::retrieve() in one call. This is the
.       most convenient method for reading video files or capturing data from decode and returns the just
.       grabbed frame. If no frames has been grabbed (camera has been disconnected, or there are no more
.       frames in video file), the method returns false and the function returns empty image (with %cv::Mat, test it with Mat::empty()).
.   
.       @note In @ref videoio_c "C API", functions cvRetrieveFrame() and cv.RetrieveFrame() return image stored inside the video
.       capturing structure. It is not allowed to modify or release the image! You can copy the frame using
.       cvClon

In [6]:
model = torch.hub.load("ultralytics/yolov5", "yolov5l6", pretrained=True)

Using cache found in C:\Users\Wabinab/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-7-18 torch 1.9.0+cu111 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264.0MB)

Fusing layers... 
C:\Users\Wabinab\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 501 layers, 77218620 parameters, 0 gradients
Adding AutoShape... 


In [111]:
cv2.namedWindow("preview")
vc = cv2.VideoCapture(0)

if vc.isOpened():
    rval, frame = vc.read()
else: 
    rval = False

for i in tqdm(range(1000)):
    results = model(frame)
    results.display(render=True)
    cv2.imshow("preview", frame)
    rval, frame = vc.read()
    key = cv2.waitKey(20)
    if key == 27:
        break
        
vc.release()
cv2.destroyWindow("preview")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:18<00:00, 12.70it/s]


In [10]:
results = model(frame)

In [16]:
model_name = "Efficientdet D0"

In [ ]:
from efficientdet.utils import BBoxTransform, ClipBoxes
from backbone import EfficientDetBackbone
from myutils.myutils import invert_affine, postprocess, preprocess_video
compound_coef = int(model_name[-1])
obj_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
            'fire hydrant', '', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
            'cow', 'elephant', 'bear', 'zebra', 'giraffe', '', 'backpack', 'umbrella', '', '', 'handbag', 'tie',
            'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
            'skateboard', 'surfboard', 'tennis racket', 'bottle', '', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
            'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
            'cake', 'chair', 'couch', 'potted plant', 'bed', '', 'dining table', '', '', 'toilet', '', 'tv',
            'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
            'refrigerator', '', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
            'toothbrush']
model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list)).cuda()
try:
    model.load_state_dict(torch.load(f'weights/efficientdet-d{compound_coef}.pth'))
except FileNotFoundError:
    os.system(f"wget https://github.com/zylo117/Yet-Another-Efficient-Pytorch/releases/download/1.0/efficientdet-d{compound_coef}.pth -P weights/")
    model.load_state_dict(torch.load(f'weights/efficientdet-d{compound_coef}.pth'))
model.requires_grad_(False)
model.eval()

In [60]:
%timeit results = model(frame)

18.3 ms ± 629 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
%timeit np.array(results.imgs).squeeze()

41.4 µs ± 3.52 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Check efficientdet 

In [24]:
%timeit ori_imgs, framed_imgs, framed_metas = preprocess_video(frame)
ori_imgs, framed_imgs, framed_metas = preprocess_video(frame)

15.3 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
use_float16 = False

In [30]:
%%timeit
x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

1.44 ms ± 23.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [32]:
x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

In [242]:
x.shape

torch.Size([1, 3, 512, 512])

In [33]:
%%timeit 
with torch.no_grad():
    features, regression, classification, anchors = model(x)

48 ms ± 5.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from line_profiler import LineProfiler

lprof = LineProfiler()

lp_wrapper = lprof(lambda: preprocess_video(frame))

lp_wrapper()

In [61]:
max_size=512
mean=(0.406, 0.456, 0.485)
std=(0.225, 0.224, 0.229)

In [62]:
%timeit normalized_imgs = [(img[..., ::-1] / 255 - mean) / std for img in ori_imgs]

13.3 ms ± 466 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
from myutils.myutils import aspectaware_resize_padding

In [66]:
normalized_imgs = [(img[..., ::-1] / 255 - mean) / std for img in ori_imgs]

In [209]:
%timeit m[:50, :50] = torch.randn((50, 50, 3), device="cuda")

158 µs ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
mean

(0.406, 0.456, 0.485)

In [87]:
%timeit torch_ori_imgs = torch.from_numpy(ori_imgs[0])

1.23 µs ± 10 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [91]:
framed_imgs[0].shape

(512, 512, 3)

In [110]:
mean=np.array([0.406, 0.456, 0.485])
std=np.array([0.225, 0.224, 0.229])

In [111]:
%timeit (ori_imgs[0][..., ::-1] / 255 - mean) / std

12.6 ms ± 344 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [116]:
torch_ori_imgs = torch.from_numpy(ori_imgs[0]).cuda()

In [117]:
torch_mean = torch.from_numpy(mean).cuda()
torch_std = torch.from_numpy(std).cuda()

In [124]:
old_h, old_w, c = torch_ori_imgs.shape

In [132]:
canvas = np.zeros((512, 512, 3), np.float32)

In [140]:
canvas = np.random.random((512, 512, 3))

In [141]:
import torch.nn.functional as F
import torchvision.transforms as T

In [147]:
transforms = torch.nn.Sequential(
    T.Resize(512)
)

In [154]:
torch_ori_imgs.shape

torch.Size([480, 640, 3])

In [161]:
F.interpolate(torch_ori_imgs, size=).shape

torch.Size([480, 640, 3])

In [168]:
torch_ori_imgs.unsqueeze(0).permute(0, 3, 2, 1).shape

torch.Size([1, 3, 640, 480])

In [ ]:
plt.imshow(cv2.resize(ori_imgs[0], (512, 512)))

In [ ]:
float_torch_ori = torch_ori_imgs / 255
float_torch_ori

In [201]:
m = torch.from_numpy(canvas)

In [203]:
m[:50, :50, 0]

tensor([[0.53530, 0.25055, 0.84055,  ..., 0.40707, 0.86913, 0.82570],
        [0.12625, 0.53034, 0.51621,  ..., 0.73781, 0.33476, 0.31189],
        [0.47031, 0.84016, 0.81861,  ..., 0.02781, 0.78931, 0.50137],
        ...,
        [0.04713, 0.31874, 0.47559,  ..., 0.46981, 0.46524, 0.30992],
        [0.74555, 0.08535, 0.62553,  ..., 0.11643, 0.57469, 0.49025],
        [0.19066, 0.25523, 0.64840,  ..., 0.51801, 0.68349, 0.32455]], dtype=torch.float64)

In [206]:
from myutils.myutils import torch_preprocess_video

ImportError: cannot import name 'torch_preprocess_video' from 'myutils.myutils' (d:\Learning\scientificProject\ML\OwnProjects\eyemain\myutils\myutils.py)

In [ ]:
temp_shape = F.interpolate(float_torch_ori.unsqueeze(0).permute(0, 3, 2, 1), size=(512, 512), mode="bilinear").squeeze().permute(2, 1, 0)
plt.imshow(temp_shape.cpu().numpy())

In [211]:
%timeit torch.stack([torch.from_numpy(fi).to("cuda") for fi in framed_imgs], 0)

1.46 ms ± 53.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [214]:
framed_imgs[0]

array([[[  -0.059074,    -0.42042,    -0.48864],
        [  -0.080316,    -0.37966,    -0.49025],
        [   -0.14431,    -0.36598,    -0.52102],
        ...,
        [   -0.16338,    -0.48279,     -0.9577],
        [   -0.14568,    -0.56212,     -1.0567],
        [   -0.12906,    -0.61327,     -1.1054]],

       [[   -0.12825,    -0.39881,    -0.44102],
        [   -0.11245,    -0.33808,    -0.41827],
        [   -0.14649,    -0.31346,    -0.43968],
        ...,
        [    -0.0972,    -0.47185,    -0.93094],
        [   -0.15303,    -0.57853,     -1.0647],
        [   -0.15602,    -0.62148,      -1.119]],

       [[   -0.22901,     -0.3482,    -0.34736],
        [   -0.20777,     -0.3203,    -0.33934],
        [   -0.20096,    -0.30252,    -0.35405],
        ...,
        [  -0.080861,    -0.53203,    -0.96172],
        [   -0.20477,    -0.64555,     -1.1289],
        [    -0.2704,    -0.70491,     -1.2114]],

       ...,

       [[          0,           0,           0],
        [  

In [171]:
%timeit F.interpolate(torch_ori_imgs.unsqueeze(0).permute(0, 3, 2, 1), size=(512, 512)).squeeze().shape

50 µs ± 3.86 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [164]:
cv2.resize(torch_ori_imgs, (512, 512))

error: OpenCV(4.5.3) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [163]:
T.Resize(512)(torch_ori_imgs)

RuntimeError: CUDA out of memory. Tried to allocate 100.00 GiB (GPU 0; 11.00 GiB total capacity; 207.34 MiB already allocated; 9.17 GiB free; 242.00 MiB reserved in total by PyTorch)

In [120]:
(torch_ori_imgs[:, :, [2, 1, 0]] / 255 - torch_mean) / torch_std

tensor([[[-0.06152, -0.42507, -0.49105],
         [-0.04410, -0.39006, -0.47393],
         [-0.14867, -0.37255, -0.52530],
         ...,
         [-0.20096, -0.53011, -1.00479],
         [-0.11381, -0.58263, -1.09042],
         [-0.13124, -0.61765, -1.10754]],

        [[-0.06152, -0.42507, -0.49105],
         [-0.02667, -0.37255, -0.45680],
         [-0.11381, -0.32003, -0.49105],
         ...,
         [-0.18353, -0.51260, -0.97055],
         [-0.11381, -0.58263, -1.09042],
         [-0.13124, -0.61765, -1.10754]],

        [[-0.25325, -0.37255, -0.37118],
         [-0.20096, -0.32003, -0.31980],
         [-0.20096, -0.30252, -0.35405],
         ...,
         [-0.13124, -0.58263, -1.02192],
         [-0.20096, -0.63515, -1.14179],
         [-0.20096, -0.63515, -1.14179]],

        ...,

        [[-0.91556, -1.14286, -1.48429],
         [-0.79355, -1.02031, -1.36441],
         [-0.77612, -1.02031, -1.22742],
         ...,
         [-1.22928, -1.68557, -2.10078],
         [-1.28157, -1

In [119]:
%timeit (torch_ori_imgs[:, :, [2, 1, 0]] / 255 - torch_mean) / torch_std

350 µs ± 65.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [67]:
%timeit imgs_meta = [aspectaware_resize_padding(img, max_size, max_size, means=None) for img in normalized_imgs]

3.76 ms ± 367 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [68]:
imgs_meta = [aspectaware_resize_padding(img, max_size, max_size, means=None) for img in normalized_imgs]

In [69]:
%%timeit 
framed_imgs = [img_meta[0] for img_meta in imgs_meta]
framed_metas = [img_meta[1:] for img_meta in imgs_meta]

565 ns ± 16.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [58]:
lprof.print_stats()

Timer unit: 1e-07 s

Total time: 0.0151638 s
File: <ipython-input-57-8e085f0d9c02>
Function: <lambda> at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5         1     151638.0 151638.0    100.0  lp_wrapper = lprof(lambda: preprocess_video(frame))



In [35]:
regressBoxes = BBoxTransform()
clipBoxes = ClipBoxes()
threshold = 0.2
iou_threshold = 0.2
with torch.no_grad():
    features, regression, classification, anchors = model(x)

In [223]:
min(torch.ceil(torch.tensor(0.35)), 3)

tensor(1.)

In [230]:
torch.tensor([0.35]).numpy()[0]

0.35

In [238]:
torch.tensor(0.35, dtype=torch.float32).item()

0.3499999940395355

In [219]:
import math
math.ceil(0.35) - 1

0

In [250]:
%%timeit -n1000
with torch.no_grad():
    frame = postprocess(x, anchors, regression, classification, regressBoxes, clipBoxes, threshold, iou_threshold)

2.63 ms ± 76.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [40]:
with torch.no_grad():
    frame = postprocess(x, anchors, regression, classification, regressBoxes, clipBoxes, threshold, iou_threshold)

class Colors:
    # Ultralytics color palette https://ultralytics.com/
    def __init__(self):
        # hex = matplotlib.colors.TABLEAU_COLORS.values()
        hex = ('FF3838', 'FF9D97', 'FF701F', 'FFB21D', 'CFD231', '48F90A', '92CC17', '3DDB86', '1A9334', '00D4BB',
               '2C99A8', '00C2FF', '344593', '6473FF', '0018EC', '8438FF', '520085', 'CB38FF', 'FF95C8', 'FF37C7')
        self.palette = [self.hex2rgb('#' + c) for c in hex]
        self.n = len(self.palette)

    def __call__(self, i, bgr=False):
        c = self.palette[int(i) % self.n]
        return (c[2], c[1], c[0]) if bgr else c

    @staticmethod
    def hex2rgb(h):  # rgb order (PIL)
        return tuple(int(h[1 + i:1 + i + 2], 16) for i in (0, 2, 4))


colors = Colors()  # create instance for 'from utils.plots import colors'

def display(preds, imgs):
    for i in range(len(imgs)):
        if len(preds[i]['rois']) == 0:
            return imgs[i]

        for j in range(len(preds[i]['rois'])):
            (x1, y1, x2, y2) = preds[i]['rois'][j].astype(np.int)
            cv2.rectangle(imgs[i], (x1, y1), (x2, y2), colors(j), 2)
            obj = obj_list[preds[i]['class_ids'][j]]
            score = float(preds[i]['scores'][j])

            cv2.putText(imgs[i], f"{obj}: {score:.3f}",
                        (x1 + 3, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        colors(j), 1, lineType=cv2.LINE_AA)

        return imgs[i]

In [43]:
%%timeit 
out = invert_affine(framed_metas, frame)
out = display(out, ori_imgs)

<ipython-input-40-dcb695063206>:36: RuntimeWarning: overflow encountered in long_scalars
  (x1 + 3, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
d:\Learning\scientificProject\ML\OwnProjects\eyemain\myutils\myutils.py:30: RuntimeWarning: overflow encountered in true_divide
  preds[i]['rois'][:, [0, 2]] = preds[i]['rois'][:, [0, 2]] / (new_w / old_w)
d:\Learning\scientificProject\ML\OwnProjects\eyemain\myutils\myutils.py:31: RuntimeWarning: overflow encountered in true_divide
  preds[i]['rois'][:, [1, 3]] = preds[i]['rois'][:, [1, 3]] / (new_h / old_h)


49 µs ± 1.89 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [245]:
anchors.shape

torch.Size([1, 49104, 4])

# Breakpoint

In [6]:
results = model(frame)

In [18]:
cv2.namedWindow("preview")
cv2.imshow("preview", results.imgs[0])

In [27]:
while True:
    results = model(frame)
    cv2.imshow("image", frame)  # results.imgs[0]
    cv2.waitKey(20)

KeyboardInterrupt: 

In [28]:
cv2.destroyWindow("image")

In [15]:
type(frame)

numpy.ndarray

In [16]:
type(results.imgs[0])

numpy.ndarray

In [69]:
np.count_nonzero(results.imgs[0] == frame)

14875

In [85]:
results.display()

In [116]:
results = model(frame)

TypeError: 'collections.OrderedDict' object is not callable

In [99]:
results.display(render=True)

In [117]:
import tkinter as tk

In [118]:
window = tk.Tk()

In [120]:
??window.mainloop

Signature: window.mainloop(n=0)
Source:   
    def mainloop(self, n=0):
        """Call the mainloop of Tk."""
        self.tk.mainloop(n)
File:      c:\users\wabinab\appdata\local\programs\python\python38\lib\tkinter\__init__.py
Type:      method


In [128]:
from PIL import ImageTk, Image

In [129]:
frame = cv2.flip(frame, 1)
cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
img = Image.fromarray(cv2image)

In [135]:
??tk.Label.after

Signature: tk.Label.after(self, ms, func=None, *args)
Source:   
    def after(self, ms, func=None, *args):
        """Call function once after given time.

        MS specifies the time in milliseconds. FUNC gives the
        function which shall be called. Additional parameters
        are given as parameters to the function call.  Return
        identifier to cancel scheduling with after_cancel."""
        if not func:
            # I'd rather use time.sleep(ms*0.001)
            self.tk.call('after', ms)
            return None
        else:
            def callit():
                try:
                    func(*args)
                finally:
                    try:
                        self.deletecommand(name)
                    except TclError:
                        pass
            callit.__name__ = func.__name__
            name = self._register(callit)
            return self.tk.call('after', ms, name)
File:      c:\users\wabinab\appdata\local\programs\python\python38\

In [141]:
??tk.OptionMenu

Init signature: tk.OptionMenu(master, variable, value, *values, **kwargs)
Source:        
class OptionMenu(Menubutton):
    """OptionMenu which allows the user to select a value from a menu."""

    def __init__(self, master, variable, value, *values, **kwargs):
        """Construct an optionmenu widget with the parent MASTER, with
        the resource textvariable set to VARIABLE, the initially selected
        value VALUE, the other menu values VALUES and an additional
        keyword argument command."""
        kw = {"borderwidth": 2, "textvariable": variable,
              "indicatoron": 1, "relief": RAISED, "anchor": "c",
              "highlightthickness": 2}
        Widget.__init__(self, master, "menubutton", kw)
        self.widgetName = 'tk_optionMenu'
        menu = self.__menu = Menu(self, name="menu", tearoff=0)
        self.menuname = menu._w
        # 'command' is the only supported keyword
        callback = kwargs.get('command')
        if 'command' in kwargs:
        